In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout,Conv3D,Conv1D,Conv2D,Flatten,MaxPool2D
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/camel-1.6.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

name  version                                             name.1  wmc  \
0    camel      1.6        org.apache.camel.component.cxf.CxfComponent    5   
1    camel      1.6     org.apache.camel.impl.DefaultExchangeFormatter    4   
2    camel      1.6  org.apache.camel.component.jetty.JettyHttpComp...   20   
3    camel      1.6    org.apache.camel.language.simple.SimpleLanguage    3   
4    camel      1.6                org.apache.camel.util.ServiceHelper    8   
..     ...      ...                                                ...  ...   
960  camel      1.6     org.apache.camel.util.concurrent.CountingLatch    6   
961  camel      1.6          org.apache.camel.component.mail.MailUtils    3   
962  camel      1.6  org.apache.camel.component.uface.swing.SwingBr...    3   
963  camel      1.6        org.apache.camel.component.jpa.QueryBuilder   12   
964  camel      1.6  org.apache.camel.processor.interceptor.Excepti...    4   

     dit  noc  cbo  rfc  lcom  ca  ...  dam  moa       mfa       cam  ic  cbm  \
0      3    0    7   10     0   1  ...  1.0    1  0.921053  0.360000   1    2   
1      1    0    3    5     4   1  ...  1.0    1  0.000000  0.500000   0    0   
2      4    0   26   95   144   2  ...  1.0    0  0.727273  0.197368   4    5   
3      2    0    8   22     3   2  ...  0.0    0  0.750000  0.666667   1    3   
4      1    0   25   20    22  22  ...  1.0    0  0.000000  0.250000   0    0   
..   ...  ...  ...  ...   ...  ..  ...  ...  ...       ...       ...  ..  ...   
960    1    0    4   13     0   3  ...  1.0    1  0.000000  0.444444   0    0   
961    1    0    4   19     3   4  ...  0.0    0  0.000000  0.333333   0    0   
962    2    0    8   16     3   1  ...  0.0    0  0.875000  0.750000   0    0   
963    1    3    8   25    46   6  ...  0.0    1  0.000000  0.226190   0    0   
964    1    0    2    5     0   1  ...  1.0    2  0.000000  0.437500   0    0   

           amc  max_cc  avg_cc  bug  
0     7.400000       1  0.6000    0  
1     3.000000       1  0.5000    0  
2    20.300000       3  1.0000    0  
3    54.000000      15  5.3333    1  
4    20.875000       1  0.7500    1  
..         ...     ...     ...  ...  
960   6.500000       1  0.8333    1  
961  61.333333       9  5.6667    0  
962  13.666667       1  0.3333    0  
963   7.416667       2  1.1667    0  
964   5.250000       1  0.7500    0  

[965 rows x 24 columns]

In [4]:
train, test = train_test_split(ant_master, test_size=0.2)

In [5]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [6]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc  dam  moa  \
71     4    1    0    6    4     6   5   2    4  1.333333    5  0.0    0   
688    2    3    0    8   11     0   1   8    2  0.000000   37  1.0    1   
208    5    2    0    9   10    10   3   6    5  2.000000   30  0.0    0   
322   13    1    0   38   76    18   3  38    9  0.766667  264  1.0    3   
885    5    1    0    3   10     8   1   2    5  0.500000   57  1.0    0   
..   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...  ...  ...   
932   18    1    0   10   56   137   0  10   15  0.935294  319  1.0    0   
571    5    3    0    7   16     4   1   7    4  0.500000   57  1.0    1   
456    3    2    0    1    5     3   0   1    3  2.000000   12  0.0    0   
175    4    3    0   24   27     4   2  24    3  0.904762  156  1.0    2   
749    2    1    0    6    2     1   2   4    2  2.000000    2  0.0    0   

          mfa       cam  ic  cbm        amc  max_cc  avg_cc  
71   0.000000  0.562500   0    0   0.000000       1  1.0000  
688  0.954545  0.666667   0    0  17.000000       1  0.5000  
208  0.926829  0.360000   1    1   5.000000       1  0.6000  
322  0.000000  0.333333   0    0  18.923077       1  0.8462  
885  0.000000  0.300000   0    0  10.200000       2  1.0000  
..        ...       ...  ..  ...        ...     ...     ...  
932  0.000000  0.176471   0    0  16.166667       3  1.0556  
571  0.840000  0.533333   0    0  10.200000       1  0.8000  
456  0.956522  0.666667   1    2   3.000000       1  0.3333  
175  0.913043  0.583333   0    0  36.250000       3  1.0000  
749  0.000000  0.666667   0    0   0.000000       1  1.0000  

[772 rows x 20 columns]

In [7]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc  dam  moa  \
801    1    1    0    6    1     0   4   2    1  2.000000    1  0.0    0   
559    2    1    0    0    2     1   0   0    2  2.000000    2  0.0    0   
178    2    1    0   12   15     0   2  10    2  0.000000   70  1.0    3   
103    9    1    0    7    9    36   4   3    9  2.000000    9  0.0    0   
78     3    3    0   10   25     0   1  10    2  0.000000  136  1.0    0   
..   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...  ...  ...   
880   20    1    0   33   96   122   6  27    8  0.726316  930  1.0    4   
718   10    1    0    7   18    43   3   4   10  0.111111   63  1.0    1   
3      3    2    0    8   22     3   2   6    2  2.000000  165  0.0    0   
682    5    1    0   12   26     2   2  10    3  0.583333  154  1.0    0   
462    5    0    2    9   22    10   2   7    1  2.000000  145  0.0    0   

          mfa       cam  ic  cbm   amc  max_cc  avg_cc  
801  0.000000  1.000000   0    0   0.0       1  1.0000  
559  0.000000  1.000000   0    0   0.0       1  1.0000  
178  0.000000  0.571429   0    0  32.5       1  0.5000  
103  0.000000  0.311111   0    0   0.0       1  1.0000  
78   0.945946  0.500000   1    1  44.0       2  1.0000  
..        ...       ...  ..  ...   ...     ...     ...  
880  0.000000  0.244019   0    0  45.0       9  3.2500  
718  0.000000  0.320000   0    0   5.2       1  0.9000  
3    0.750000  0.666667   1    3  54.0      15  5.3333  
682  0.571429  0.500000   1    1  29.2       1  0.6000  
462  0.000000  0.500000   0    0  28.0       4  1.8000  

[193 rows x 20 columns]

In [8]:
y_train = pd.DataFrame(train['bug'])
y_test = pd.DataFrame(test['bug'])

In [9]:
y_train

bug
71     0
688    0
208    0
322    0
885    0
..   ...
932    0
571    0
456    0
175    0
749    0

[772 rows x 1 columns]

In [10]:
y_test

bug
801    0
559    0
178    1
103    0
78     0
..   ...
880    4
718    0
3      1
682    0
462    0

[193 rows x 1 columns]

In [11]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,20
# Preparing the data for the model.
X_train_matrix = x_train.values
X_test_matrix = x_test.values
Y_train_matrix = y_train.values
Y_test_matrix = y_test.values
# Preparing the data for the model.
Ytrainseries = y_train['bug']
Ytestseries = y_test['bug']

In [12]:
# Preparing the data for the model.
X_train1 = X_train_matrix.reshape(X_train_matrix.shape[0], img_rows, img_cols, 1)
X_test1 = X_test_matrix.reshape(X_test_matrix.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [13]:
def define_model():
    # Designing and initializing the model.
    #Building the model
    model = Sequential()
    #add model layers
    model.add(Conv1D(2, kernel_size=2, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation='relu'))
    #compile model using mse as the loss function
    model.compile(loss='mse', optimizer = keras.optimizers.Adam(learning_rate=0.0005), metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])
    return model

In [14]:
y = y_test['bug'].tolist()
y

[0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 6,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 4,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 2,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 7,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 10,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 1,
 0,
 0]

In [15]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(30):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/cnn_camel1.6/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(X_train1, Y_train_matrix, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(X_test1, Y_test_matrix)
    total_scores.append(score) 
    predictions = model.predict(X_test1)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
8/8 [==============================] - 2s 15ms/step - loss: 716.4673 - mse: 716.4673 - mae: 5.9296 - root_mean_squared_error: 26.7669 - mean_squared_logarithmic_error: 1.3241
Epoch 2/100
8/8 [==============================] - 0s 4ms/step - loss: 496.0363 - mse: 496.0363 - mae: 4.6357 - root_mean_squared_error: 22.2719 - mean_squared_logarithmic_error: 1.0576
Epoch 3/100
8/8 [==============================] - 0s 5ms/step - loss: 349.5610 - mse: 349.5610 - mae: 3.6332 - root_mean_squared_error: 18.6966 - mean_squared_logarithmic_error: 0.8328
Epoch 4/100
8/8 [==============================] - 0s 4ms/step - loss: 257.5276 - mse: 257.5276 - mae: 2.8052 - root_mean_squared_error: 16.0477 - mean_squared_logarithmic_error: 0.6590
Epoch 5/100
8/8 [==============================] - 0s 4ms/step - loss: 160.4587 - mse: 160.4587 - mae: 2.1089 - root_mean_squared_error: 12.6672 - mean_squared_logarithmic_error: 0.5355
Epoch 6/100
8/8 [==============================] - 0s 4ms/s

8/8 [==============================] - 0s 4ms/step - loss: 3.9530 - mse: 3.9530 - mae: 0.5345 - root_mean_squared_error: 1.9882 - mean_squared_logarithmic_error: 0.2872
Epoch 90/100
8/8 [==============================] - 0s 5ms/step - loss: 3.9945 - mse: 3.9945 - mae: 0.5358 - root_mean_squared_error: 1.9986 - mean_squared_logarithmic_error: 0.2884
Epoch 91/100
8/8 [==============================] - 0s 4ms/step - loss: 3.9414 - mse: 3.9414 - mae: 0.5337 - root_mean_squared_error: 1.9853 - mean_squared_logarithmic_error: 0.2873
Epoch 92/100
8/8 [==============================] - 0s 5ms/step - loss: 3.9439 - mse: 3.9439 - mae: 0.5340 - root_mean_squared_error: 1.9859 - mean_squared_logarithmic_error: 0.2869
Epoch 93/100
8/8 [==============================] - 0s 5ms/step - loss: 3.9730 - mse: 3.9730 - mae: 0.5358 - root_mean_squared_error: 1.9932 - mean_squared_logarithmic_error: 0.2866
Epoch 94/100
8/8 [==============================] - 0s 5ms/step - loss: 3.9646 - mse: 3.9646 - mae: 0.5

8/8 [==============================] - 0s 4ms/step - loss: 3.8041 - mse: 3.8041 - mae: 0.5199 - root_mean_squared_error: 1.9504 - mean_squared_logarithmic_error: 0.2831
Epoch 77/100
8/8 [==============================] - 0s 4ms/step - loss: 3.8040 - mse: 3.8040 - mae: 0.5199 - root_mean_squared_error: 1.9504 - mean_squared_logarithmic_error: 0.2831
Epoch 78/100
8/8 [==============================] - 0s 4ms/step - loss: 3.8034 - mse: 3.8034 - mae: 0.5198 - root_mean_squared_error: 1.9502 - mean_squared_logarithmic_error: 0.2831
Epoch 79/100
8/8 [==============================] - 0s 4ms/step - loss: 3.8030 - mse: 3.8030 - mae: 0.5197 - root_mean_squared_error: 1.9501 - mean_squared_logarithmic_error: 0.2831
Epoch 80/100
8/8 [==============================] - 0s 5ms/step - loss: 3.8023 - mse: 3.8023 - mae: 0.5196 - root_mean_squared_error: 1.9499 - mean_squared_logarithmic_error: 0.2831
Epoch 81/100
8/8 [==============================] - 0s 4ms/step - loss: 3.8018 - mse: 3.8018 - mae: 0.5

8/8 [==============================] - 0s 4ms/step - loss: 572.3038 - mse: 572.3038 - mae: 5.8433 - root_mean_squared_error: 23.9229 - mean_squared_logarithmic_error: 1.5476
Epoch 61/100
8/8 [==============================] - 0s 5ms/step - loss: 554.3540 - mse: 554.3540 - mae: 5.6789 - root_mean_squared_error: 23.5447 - mean_squared_logarithmic_error: 1.4992
Epoch 62/100
8/8 [==============================] - 0s 4ms/step - loss: 534.2523 - mse: 534.2523 - mae: 5.5187 - root_mean_squared_error: 23.1139 - mean_squared_logarithmic_error: 1.4536
Epoch 63/100
8/8 [==============================] - 0s 4ms/step - loss: 516.8438 - mse: 516.8438 - mae: 5.3674 - root_mean_squared_error: 22.7342 - mean_squared_logarithmic_error: 1.4112
Epoch 64/100
8/8 [==============================] - 0s 4ms/step - loss: 498.3658 - mse: 498.3659 - mae: 5.2199 - root_mean_squared_error: 22.3241 - mean_squared_logarithmic_error: 1.3698
Epoch 65/100
8/8 [==============================] - 0s 4ms/step - loss: 483.61

8/8 [==============================] - 0s 4ms/step - loss: 4.7877 - mse: 4.7877 - mae: 0.5513 - root_mean_squared_error: 2.1881 - mean_squared_logarithmic_error: 0.2931
Epoch 47/100
8/8 [==============================] - 0s 5ms/step - loss: 4.7688 - mse: 4.7688 - mae: 0.5502 - root_mean_squared_error: 2.1838 - mean_squared_logarithmic_error: 0.2930
Epoch 48/100
8/8 [==============================] - 0s 5ms/step - loss: 4.7537 - mse: 4.7537 - mae: 0.5488 - root_mean_squared_error: 2.1803 - mean_squared_logarithmic_error: 0.2928
Epoch 49/100
8/8 [==============================] - 0s 5ms/step - loss: 4.7334 - mse: 4.7334 - mae: 0.5478 - root_mean_squared_error: 2.1756 - mean_squared_logarithmic_error: 0.2928
Epoch 50/100
8/8 [==============================] - 0s 4ms/step - loss: 4.7226 - mse: 4.7226 - mae: 0.5468 - root_mean_squared_error: 2.1732 - mean_squared_logarithmic_error: 0.2927
Epoch 51/100
8/8 [==============================] - 0s 4ms/step - loss: 4.7066 - mse: 4.7066 - mae: 0.5

...

8/8 [==============================] - 0s 3ms/step - loss: 12.9495 - mse: 12.9495 - mae: 0.9220 - root_mean_squared_error: 3.5985 - mean_squared_logarithmic_error: 0.3956
Epoch 96/100
8/8 [==============================] - 0s 3ms/step - loss: 12.2427 - mse: 12.2427 - mae: 0.8985 - root_mean_squared_error: 3.4990 - mean_squared_logarithmic_error: 0.3875
Epoch 97/100
8/8 [==============================] - 0s 3ms/step - loss: 11.5730 - mse: 11.5730 - mae: 0.8766 - root_mean_squared_error: 3.4019 - mean_squared_logarithmic_error: 0.3794
Epoch 98/100
8/8 [==============================] - 0s 3ms/step - loss: 10.8843 - mse: 10.8843 - mae: 0.8533 - root_mean_squared_error: 3.2991 - mean_squared_logarithmic_error: 0.3717
Epoch 99/100
8/8 [==============================] - 0s 3ms/step - loss: 10.2928 - mse: 10.2928 - mae: 0.8349 - root_mean_squared_error: 3.2082 - mean_squared_logarithmic_error: 0.3670
Epoch 100/100
7/7 [==============================] - 0s 2ms/step - loss: 248.6621 - mse: 248.

8/8 [==============================] - 0s 2ms/step - loss: 8.5237 - mse: 8.5237 - mae: 1.2524 - root_mean_squared_error: 2.9195 - mean_squared_logarithmic_error: 0.6083
Epoch 81/100
8/8 [==============================] - 0s 2ms/step - loss: 8.1362 - mse: 8.1362 - mae: 1.2173 - root_mean_squared_error: 2.8524 - mean_squared_logarithmic_error: 0.5925
Epoch 82/100
8/8 [==============================] - 0s 3ms/step - loss: 7.7636 - mse: 7.7636 - mae: 1.1856 - root_mean_squared_error: 2.7863 - mean_squared_logarithmic_error: 0.5792
Epoch 83/100
8/8 [==============================] - 0s 3ms/step - loss: 7.4625 - mse: 7.4625 - mae: 1.1548 - root_mean_squared_error: 2.7318 - mean_squared_logarithmic_error: 0.5658
Epoch 84/100
8/8 [==============================] - 0s 2ms/step - loss: 7.1613 - mse: 7.1613 - mae: 1.1252 - root_mean_squared_error: 2.6761 - mean_squared_logarithmic_error: 0.5533
Epoch 85/100
8/8 [==============================] - 0s 3ms/step - loss: 6.8766 - mse: 6.8766 - mae: 1.0

8/8 [==============================] - 0s 3ms/step - loss: 3.7630 - mse: 3.7630 - mae: 0.5142 - root_mean_squared_error: 1.9398 - mean_squared_logarithmic_error: 0.2881
Epoch 68/100
8/8 [==============================] - 0s 3ms/step - loss: 3.7630 - mse: 3.7630 - mae: 0.5142 - root_mean_squared_error: 1.9398 - mean_squared_logarithmic_error: 0.2881
Epoch 69/100
8/8 [==============================] - 0s 4ms/step - loss: 3.7630 - mse: 3.7630 - mae: 0.5142 - root_mean_squared_error: 1.9398 - mean_squared_logarithmic_error: 0.2881
Epoch 70/100
8/8 [==============================] - 0s 4ms/step - loss: 3.7630 - mse: 3.7630 - mae: 0.5142 - root_mean_squared_error: 1.9398 - mean_squared_logarithmic_error: 0.2881
Epoch 71/100
8/8 [==============================] - 0s 3ms/step - loss: 3.7630 - mse: 3.7630 - mae: 0.5142 - root_mean_squared_error: 1.9398 - mean_squared_logarithmic_error: 0.2881
Epoch 72/100
8/8 [==============================] - 0s 3ms/step - loss: 3.7630 - mse: 3.7630 - mae: 0.5

8/8 [==============================] - 0s 3ms/step - loss: 23.7141 - mse: 23.7141 - mae: 2.1272 - root_mean_squared_error: 4.8697 - mean_squared_logarithmic_error: 1.0367
Epoch 54/100
8/8 [==============================] - 0s 3ms/step - loss: 22.1873 - mse: 22.1873 - mae: 2.0333 - root_mean_squared_error: 4.7103 - mean_squared_logarithmic_error: 0.9941
Epoch 55/100
8/8 [==============================] - 0s 3ms/step - loss: 20.8344 - mse: 20.8344 - mae: 1.9473 - root_mean_squared_error: 4.5645 - mean_squared_logarithmic_error: 0.9568
Epoch 56/100
8/8 [==============================] - 0s 2ms/step - loss: 19.4414 - mse: 19.4414 - mae: 1.8607 - root_mean_squared_error: 4.4092 - mean_squared_logarithmic_error: 0.9193
Epoch 57/100
8/8 [==============================] - 0s 3ms/step - loss: 18.2480 - mse: 18.2480 - mae: 1.7798 - root_mean_squared_error: 4.2718 - mean_squared_logarithmic_error: 0.8855
Epoch 58/100
8/8 [==============================] - 0s 3ms/step - loss: 17.1842 - mse: 17.184

8/8 [==============================] - 0s 2ms/step - loss: 3.7630 - mse: 3.7630 - mae: 0.5142 - root_mean_squared_error: 1.9398 - mean_squared_logarithmic_error: 0.2881
Epoch 40/100
8/8 [==============================] - 0s 3ms/step - loss: 3.7630 - mse: 3.7630 - mae: 0.5142 - root_mean_squared_error: 1.9398 - mean_squared_logarithmic_error: 0.2881
Epoch 41/100
8/8 [==============================] - 0s 3ms/step - loss: 3.7630 - mse: 3.7630 - mae: 0.5142 - root_mean_squared_error: 1.9398 - mean_squared_logarithmic_error: 0.2881
Epoch 42/100
8/8 [==============================] - 0s 2ms/step - loss: 3.7630 - mse: 3.7630 - mae: 0.5142 - root_mean_squared_error: 1.9398 - mean_squared_logarithmic_error: 0.2881
Epoch 43/100
8/8 [==============================] - 0s 2ms/step - loss: 3.7630 - mse: 3.7630 - mae: 0.5142 - root_mean_squared_error: 1.9398 - mean_squared_logarithmic_error: 0.2881
Epoch 44/100
8/8 [==============================] - 0s 3ms/step - loss: 3.7630 - mse: 3.7630 - mae: 0.5

In [16]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/cnn_camel1.6/'

In [17]:
total_training_time

[6.4023613929748535,
 6.5220465660095215,
 5.626454830169678,
 5.820327520370483,
 5.995218276977539,
 5.840547323226929,
 5.897822618484497,
 6.45077657699585,
 5.98762321472168,
 6.068531274795532,
 3.580275297164917,
 4.596885681152344,
 4.300493001937866,
 4.115707159042358,
 4.837265729904175,
 4.179672718048096,
 4.3873255252838135,
 4.338305711746216,
 5.3313634395599365,
 4.474178791046143,
 4.680436134338379,
 4.21140456199646,
 4.739376783370972,
 4.236226320266724,
 4.2801291942596436,
 3.6229100227355957,
 3.678784132003784,
 4.527233839035034,
 4.296161890029907,
 4.182903528213501]

In [18]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 4.906958301862081


In [19]:
total_training_loss_history

[{'loss': [716.46728515625,
   496.0362548828125,
   349.56103515625,
   257.5275573730469,
   160.45867919921875,
   101.8034896850586,
   70.8203125,
   41.480682373046875,
   24.882230758666992,
   17.900636672973633,
   11.891206741333008,
   9.309216499328613,
   7.051504135131836,
   5.562856197357178,
   5.199856758117676,
   4.590916156768799,
   4.3894805908203125,
   4.289458751678467,
   4.212980270385742,
   4.164475917816162,
   4.08643102645874,
   4.055903911590576,
   4.034627914428711,
   3.9984517097473145,
   3.982898712158203,
   3.995120048522949,
   3.9834296703338623,
   3.993654251098633,
   3.9850480556488037,
   3.9823756217956543,
   3.9791297912597656,
   3.9770753383636475,
   3.9714605808258057,
   3.9825141429901123,
   3.9707467555999756,
   3.9759111404418945,
   3.980693817138672,
   3.9797651767730713,
   3.972980499267578,
   3.9779233932495117,
   3.9816274642944336,
   3.9663708209991455,
   3.9871301651000977,
   3.9686336517333984,
   3.962430238

In [20]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(30):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 13.510237646102905
avg mae of training: 0.7893253495295842


In [21]:
total_scores

[[783.4088134765625,
  783.4088134765625,
  3.103790760040283,
  27.98944091796875,
  0.5763171315193176],
 [2033.143798828125,
  2033.143798828125,
  4.868136882781982,
  45.090396881103516,
  0.6807567477226257],
 [10946.943359375,
  10946.943359375,
  13.241086959838867,
  104.62763977050781,
  1.4932410717010498],
 [1664.1580810546875,
  1664.1580810546875,
  4.344334602355957,
  40.79409408569336,
  0.6229481101036072],
 [971.1297607421875,
  971.1297607421875,
  3.850249767303467,
  31.162954330444336,
  0.7781756520271301],
 [1753.6767578125,
  1753.6767578125,
  4.891390800476074,
  41.876922607421875,
  0.7920861840248108],
 [122.31131744384766,
  122.31131744384766,
  2.52704119682312,
  11.059444427490234,
  0.8923388719558716],
 [1818.625,
  1818.625,
  4.676591873168945,
  42.64533996582031,
  0.6855936050415039],
 [3935.57666015625,
  3935.57666015625,
  8.708700180053711,
  62.73417282104492,
  1.339292049407959],
 [121.77008819580078,
  121.77008819580078,
  4.482214450

In [4]:
a = [[783.4088134765625,
  783.4088134765625,
  3.103790760040283,
  27.98944091796875,
  0.5763171315193176],
 [2033.143798828125,
  2033.143798828125,
  4.868136882781982,
  45.090396881103516,
  0.6807567477226257],
 [10946.943359375,
  10946.943359375,
  13.241086959838867,
  104.62763977050781,
  1.4932410717010498],
 [1664.1580810546875,
  1664.1580810546875,
  4.344334602355957,
  40.79409408569336,
  0.6229481101036072],
 [971.1297607421875,
  971.1297607421875,
  3.850249767303467,
  31.162954330444336,
  0.7781756520271301],
 [1753.6767578125,
  1753.6767578125,
  4.891390800476074,
  41.876922607421875,
  0.7920861840248108],
 [122.31131744384766,
  122.31131744384766,
  2.52704119682312,
  11.059444427490234,
  0.8923388719558716],
 [1818.625,
  1818.625,
  4.676591873168945,
  42.64533996582031,
  0.6855936050415039],
 [3935.57666015625,
  3935.57666015625,
  8.708700180053711,
  62.73417282104492,
  1.339292049407959],
 [121.77008819580078,
  121.77008819580078,
  4.482214450836182,
  11.034948348999023,
  1.7547301054000854],
 [496.9274597167969,
  496.9274597167969,
  2.681605100631714,
  22.2918701171875,
  0.5951322913169861],
 [3.1856892108917236,
  3.1856892108917236,
  0.6494730710983276,
  1.7848498821258545,
  0.3630310893058777],
 [2.2055299282073975,
  2.2055299282073975,
  0.6584745645523071,
  1.485102653503418,
  0.27325525879859924],
 [97.82772827148438,
  97.82772827148438,
  1.5574699640274048,
  9.890789985656738,
  0.49536097049713135],
 [291.81695556640625,
  291.81695556640625,
  2.220767021179199,
  17.082651138305664,
  0.5144612789154053],
 [354.1169128417969,
  354.1169128417969,
  2.5199859142303467,
  18.817995071411133,
  0.589576244354248],
 [2.544041395187378,
  2.544041395187378,
  0.5336787700653076,
  1.5950051546096802,
  0.314142644405365],
 [2.544041395187378,
  2.544041395187378,
  0.5336787700653076,
  1.5950051546096802,
  0.314142644405365],
 [71.61558532714844,
  71.61558532714844,
  1.4295657873153687,
  8.46259880065918,
  0.5051748156547546],
 [2881.663330078125,
  2881.663330078125,
  5.79299783706665,
  53.68112564086914,
  0.6974683403968811],
 [2.544041395187378,
  2.544041395187378,
  0.5336787700653076,
  1.5950051546096802,
  0.314142644405365],
 [2.1505794525146484,
  2.1505794525146484,
  0.5135771036148071,
  1.4664853811264038,
  0.2880100905895233],
 [2.449388027191162,
  2.449388027191162,
  0.6212618350982666,
  1.5650520324707031,
  0.3033812642097473],
 [7.077096462249756,
  7.077096462249756,
  1.0172710418701172,
  2.660281181335449,
  0.4428947865962982],
 [1938.023193359375,
  1938.023193359375,
  4.848446846008301,
  44.02298355102539,
  0.6973389983177185],
 [248.66209411621094,
  248.66209411621094,
  2.5481536388397217,
  15.769022941589355,
  0.7621520161628723],
 [14.766077995300293,
  14.766077995300293,
  1.4517546892166138,
  3.842665433883667,
  0.7177596092224121],
 [2.544041395187378,
  2.544041395187378,
  0.5336787700653076,
  1.5950051546096802,
  0.314142644405365],
 [5.622990608215332,
  5.622990608215332,
  0.9174043536186218,
  2.3712844848632812,
  0.4719111919403076],
 [2.544041395187378,
  2.544041395187378,
  0.5336787700653076,
  1.5950051546096802,
  0.314142644405365]]

In [8]:
test_mae = []
for i in range(20,30):
    test_mae.append(a[i][2])

In [9]:
test_mae

[0.5336787700653076,
 0.5135771036148071,
 0.6212618350982666,
 1.0172710418701172,
 4.848446846008301,
 2.5481536388397217,
 1.4517546892166138,
 0.5336787700653076,
 0.9174043536186218,
 0.5336787700653076]

In [25]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(30):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 1019.385815167427
avg mae: 2.893004669745763
avg are: 1.5626978024840354


In [26]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 4.073283910751343
median testing mse: 122.04070281982422


In [27]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.5693669617176056
median testing mae: 2.370376467704773


In [28]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.8891561627388


In [29]:
test_ARE

[2.0932643,
 3.1917098,
 7.3688254,
 2.937306,
 2.2625215,
 2.9896374,
 0.92339003,
 3.0259068,
 4.5333767,
 0.6559339,
 1.6839378,
 0.22150259,
 0.19022946,
 0.8549223,
 1.3453368,
 1.4339378,
 0.20725389,
 0.20725389,
 0.746114,
 3.7642486,
 0.20725389,
 0.19948186,
 0.20392302,
 0.2674562,
 3.1088083,
 1.2452505,
 0.38903278,
 0.20725389,
 0.2086109,
 0.20725389]